#Importo y descargo lo necesario

In [1]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
from sklearn.utils import resample

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from category_encoders import CatBoostEncoder
import category_encoders as ce

#Cargo mi set de datos analizado

In [6]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/train_set_analizado.parq'

train_set = pd.read_parquet(parquet_file)

#Realizo el split

In [7]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]

Mi data_set ya se encuentra ordenado por ips por lo que unicamente divido el set y las ips no se repiten entre el train y el test

Para este baseline opté realizar el split separando ordenadamente pero no es algo que me haya funcionado para los demas modelos. La razón por la que lo dejo asi es porque no quiero alterar el baseline ya que se uso como base en gran medida para la mejora en los siguientes dos modelos

In [8]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.35, shuffle=False)

del train_set
del X
del y

Quito las columnas de attacker_ip_enum porque no quisiera que mi modelo se memorizara las IP para predecir quien es VPN y quien no

In [9]:
X_train = X_train.drop(columns=['attacker_ip_enum'])
X_test = X_test.drop(columns=['attacker_ip_enum'])

#Encodeo

Encodeo de watcher_as_name y attacker_as_name

In [10]:
encoderCatBoost_asname = CatBoostEncoder(cols=['watcher_as_name', 'attacker_as_name'])
X_train1, X_train2, y_train1, y_train2 = train_test_split( X_train, y_train, test_size=0.5, shuffle=False)

encoderCatBoost_asname.fit(X_train, y_train) #Entreno el modelo SOLO con el y_train

X_train1 = encoderCatBoost_asname.transform(X_train1, y_train1) #Aplico la transformacion parcial a la primera mitad del train
X_train2 = encoderCatBoost_asname.transform(X_train2, y_train2) #Aplico la transformacion parcial a la segunda mitad del train

del y_train1
del y_train2

X_train = pd.concat([X_train1, X_train2], axis=0, ignore_index=False)
del X_train1
del X_train2

X_test1, X_test2, y_test1, y_test2 = train_test_split( X_test, y_test, test_size=0.5, shuffle=False)

X_test1 = encoderCatBoost_asname.transform(X_test1, y_test1) #Aplico la transformacion parcial a la primera mitad del test
X_test2 = encoderCatBoost_asname.transform(X_test2, y_test2) #Aplico la transformacion parcial a la segunda mitad del test

del y_test1
del y_test2

X_test = pd.concat([X_test1, X_test2], axis=0, ignore_index=False)
del X_test1
del X_test2

Encodeo de attack_type

In [11]:
# Crear un objeto BinaryEncoder
encoder_attacktype = ce.BinaryEncoder(cols=['attack_type'])

encoder_attacktype.fit(X_train)
X_train = encoder_attacktype.transform(X_train)
X_test = encoder_attacktype.transform(X_test)

Encodeo de watcher_country y attacker_country

In [12]:
encoderCatBoost_country = CatBoostEncoder(cols=['watcher_country','attacker_country'])

# Ajusto el codificador a tus datos de entrenamiento.
encoderCatBoost_country.fit(X_train, y_train)

#Tranformo los datos de entrenamiento y de validacion.
X_train = encoderCatBoost_country.transform(X_train, y_train)
X_test = encoderCatBoost_country.transform(X_test, y_test)

#Guardo el set de train

In [13]:
set_train = pd.concat([X_train, y_train], axis=1)
set_train.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/set_train_perceptron_encoded.parq')

del X_train
del y_train
del set_train

#Guardo el set de validacion

In [14]:
set_validation = pd.concat([X_test, y_test], axis=1)
set_validation.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/set_validation_perceptron_encoded.parq')

del X_test
del y_test
del set_validation

#Kaggle_set

Cargamos el archivo de kaggle analizado

In [15]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/test_set_analizado.parq'

test_set = pd.read_parquet(parquet_file)

test_set

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum
0,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:spam,1,7696
1,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:exploit,1,7696
2,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:bruteforce,1,7696
3,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:bruteforce,1,7696
4,7.283333,US,GOOGLE-CLOUD-PLATFORM,US,PONYNET,http:bruteforce,3,7543
...,...,...,...,...,...,...,...,...
18682292,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081
18682293,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081
18682294,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081
18682295,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081


In [16]:
attacker_id = test_set['attacker_ip_enum']
test_set = test_set.drop('attacker_ip_enum', axis=1)

Encodeamos

In [17]:
test_set = encoderCatBoost_asname.transform(test_set)

test_set = encoder_attacktype.transform(test_set)

test_set = encoderCatBoost_country.transform(test_set)

In [18]:
test_set = pd.concat([test_set, attacker_id], axis=1)

test_set.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/test_kaggle_perceptron_encoded.parq')

test_set

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type_0,attack_type_1,attack_type_2,attack_type_3,watcher_uuid_enum,attacker_ip_enum
0,7.283333,0.021330,0.022318,0.010713,1.646918e-02,0,0,0,1,1,7696
1,7.283333,0.021330,0.022318,0.010713,1.646918e-02,0,1,0,0,1,7696
2,7.283333,0.021330,0.022318,0.010713,1.646918e-02,0,0,1,1,1,7696
3,7.283333,0.021330,0.022318,0.010713,1.646918e-02,0,0,1,1,1,7696
4,7.283333,0.021330,0.032953,0.024376,3.773450e-02,0,0,1,1,3,7543
...,...,...,...,...,...,...,...,...,...,...,...
18682292,21.233334,0.016079,0.006755,0.015448,9.963406e-08,0,0,1,0,24879,198081
18682293,21.233334,0.016079,0.006755,0.015448,9.963406e-08,0,0,1,0,24879,198081
18682294,21.233334,0.016079,0.006755,0.015448,9.963406e-08,0,0,1,0,24879,198081
18682295,21.233334,0.016079,0.006755,0.015448,9.963406e-08,0,0,1,0,24879,198081
